## Zadanie domowe: BBHE i DSIHE

W klasycznym wyrównywaniu histogramu HE  po wykonaniu operacji jasność obrazu ulega zmianie.
Dało się to zaobserwować podczas przeprowadzonych eksperymentów.
Jeśli nie to należy uruchomić skrypt z sekcji A i zwrócić na to uwagę.
Średnia jasność dąży do środkowego poziomu szarości.
Jest to wada i dlatego klasyczne HE ma ograniczone zastosowanie.

Powstało sporo metod, które eliminują to niekorzystne zjawisko.
Najprostsze z nich polegają na dekompozycji obrazu wejściowego na dwa podobrazy (wg. pewnego kryterium).
Następnie operacja HE wykonywana jest dla tych podobrazów.

Dwie znane z literatury metody to:
- Bi-Histogram Equalization
- DSIHE - Dualistic Sub-Image Histogram Equalization

W metodzie BBHE za kryterium podziału przyjmuje się średnią jasność w obrazie.
W DSIHE obraz dzieli się na dwa podobrazy o takiej samej liczbie pikseli (jaśniejszych i ciemniejszych).

W ramach zadania należy zaimplementować wybraną metodę: BBHE lub DSIHE (ew. obie).

1. Wczytaj obraz *jet.bmp* i wylicz jego histogram.
2. W kolejnym kroku należy wyznaczyć próg podziału obrazu na dwa podobrazy (*lm*).
3. Dla BBHE wyznacz średnią jasność obrazu. Dla DSIHE można wykorzystać histogram skumulowany.
Należy znaleźć poziom jasności który znajduje się "w połowie" histogramu skumulowanego.
W tym celu warto stworzyć tablicę, zawierającą moduł histogramu skumulowanego pomniejszonego o połowę liczby pikseli.
Następnie znaleźć minimum.
4. Dalej należy podzielić histogram oryginalnego obrazu na dwa histogramy *H1* i *H2*.
Dla każdego z nich wyliczyć histogram skumulowany ($C_1$ i $C_2$) i wykonać normalizację.
Normalizacja polega na podzieleniu każdego histogramu przez jego największy element.
5. Na podstawie histogramów skumulowanych należy stworzyć przekształcenie LUT.
Należy tak przeskalować $C_1$ i $C_2$, aby uzyskać jednorodne przekształcenie.
Tablicę $C_1$ wystarczy pomnożyć przez próg podziału.
Tablicę $C_2$ należy przeskalować do przedziału: $<lm+1; 255>$, gdzie $lm$ jest progiem podziału.<br>
$C_{1n} = (lm)*C1;$<br>
$C_{2n} = lm+1 + (255-lm-1)*C2;$<br>
Następnie dwie części tablicy przekodowań należy połączyć.
6. Ostatecznie należy wykonać operację LUT i wyświetlić wynik wyrównywania histogramu.
Porównaj wynik operacji BBHE lub DSIHE z klasycznym HE.

In [ ]:
import cv2
import os
from matplotlib import pyplot as plt
import numpy as np

if not os.path.exists("jet.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/03_Histogram/jet.bmp --no-check-certificate
jet = cv2.imread("jet.bmp")
H = cv2.calcHist([jet], [0], None, [256], [0, 256])
#dimensions of np.average op
jet_avg1 = np.uint8(np.average(jet))

figjet, axsjet = plt.subplots(2, 1)
figjet.set_size_inches(8, 10)
axsjet[0].imshow(jet)
axsjet[1].plot(H)
axsjet[1].plot(np.cumsum(H)*(max(H)/max(sum(H))))

for i in range(2):
    if i == 0:
        jet_avg = 1
    else:
        jet_avg = jet_avg1
    H1 = H[:jet_avg]
    H2 = H[jet_avg:]

    C1 = np.cumsum(H1)
    C2 = np.cumsum(H2)

    C1_norm = C1/max(C1)
    C2_norm = C2/max(C2)

    # figjet, axsjet = plt.subplots(1, 2)
    # figjet.set_size_inches(8, 10)
    # axsjet[0].plot(H1)
    # axsjet[0].plot(C1_norm)
    # axsjet[1].plot(H2)
    # axsjet[1].plot(C2_norm)

    C1n = jet_avg * C1_norm
    C2n = jet_avg + 1 + (254 - jet_avg) * C2_norm


    C_fin  = np.array(list(C1n) + list(C2n))

    # figjet, axsjet = plt.subplots()
    # figjet.set_size_inches(8, 10)
    # axsjet.plot(C_fin)
    # axsjet.plot(H*256/max(H))

    jet_BHE = cv2.LUT(jet, C_fin).astype('uint8')
    H_BHE = cv2.calcHist([jet_BHE], [0], None, [256], [0, 255]) 

    figjet, axsjet = plt.subplots(2, 1)
    axsjet[0].imshow(jet_BHE)
    axsjet[1].plot(H_BHE)
    axsjet[1].plot(np.cumsum(H_BHE)*(max(H_BHE)/max(sum(H_BHE))))




Bhe uwidacznia szczegóły gór, a zwykly lut szczegóły chmur, tj BHE w tym przypadku nie zaciemnia jasnych obszarów